# ノートブックのパラメタライズ

SSH Kernelのノートブックで変数を宣言・参照する方法を説明します。<br><br><br>

## 変数の宣言：`%param`

`%param <変数名> <変数値>` 記法は、新しい変数を宣言します。

* 注
    * 〈変数名〉に使うことのできる文字は次の通り： A-Z a-z 0-9 _
    * 〈変数値〉に使うことのできる文字は次の通り： A-Z a-z 0-9 - % , . / : = _ @ SPACE

### 例

In [1]:
%param HOST_A 127.0.0.1
%param HOST_B test.example.com

==> `HOST_A`, `HOST_B`の変数が宣言され、次に示す方法で参照できます。

<br><br><br>

## `%login`での変数展開

上で宣言された変数は、`%login` 記法のなかで `{name}` と書くことで展開できます。

### 例

In [2]:
%login {HOST_A}

[ssh] Login to {HOST_A}...
[ssh] host=127.0.0.1 hostname=127.0.0.1 other_conf={}
[ssh] Successfully logged in.


==> `{HOST_A}` は `127.0.0.1` に置き換わります

<br><br><br>

## リモート環境で環境変数として参照する

上で宣言された変数は、リモート環境で環境変数として参照することができます。

* 注
    * 宣言された変数は、環境変数のデフォルト値として、リモート環境へのログイン時に注入されます
        * ログイン後に環境変数を再代入した場合（つまりコマンド`export HOST_B=xxx`と実行した場合）には、その変数はそのリモート環境でのみ上書きされます

### 例

In [3]:
%login {HOST_B}

[ssh] Closing existing connection.
[ssh] Login to {HOST_B}...
[ssh] host=test.example.com hostname=localhost other_conf={'port': 22}
[ssh] Successfully logged in.


In [4]:
env |sort | grep HOST_

[ssh] host = test.example.com, cwd = /home/masaru
HOST_A=127.0.0.1
HOST_B=test.example.com



In [5]:
echo HOST_A is $HOST_A
echo HOST_B is $HOST_B

[ssh] host = test.example.com, cwd = /home/masaru
HOST_A is 127.0.0.1
HOST_B is test.example.com



In [6]:
%logout

[ssh] Successfully logged out.


## Papermillとの統合

[Papermill](https://github.com/nteract/papermill)に次のパッチを当てて用いると、SSH Kernel形式のノートブックをコマンドラインからパラメタライズ実行できます。

* `%param` を含むセルをノートブックに挿入するためのパッチ：

```diff
--- papermill/translators.py.orig       2019-05-10 18:44:49.592111720 +0900
+++ papermill/translators.py    2019-04-25 17:32:40.204800528 +0900
@@ -200,12 +200,31 @@
         return '# {}'.format(cmt_str).strip()


+class SSHTranslator(Translator):
+    @classmethod
+    def translate_str(cls, val):
+        """Default behavior for translation"""
+        return cls.translate_raw_str(val)
+
+    @classmethod
+    def assign(cls, name, str_val):
+        return '%param {} {}'.format(name, str_val)
+
+    @classmethod
+    def codify(cls, parameters):
+        content = ''
+        for name, val in parameters.items():
+            content += '{}\n'.format(cls.assign(name, cls.translate(val)))
+        return content
+
+
 # Instantiate a PapermillIO instance and register Handlers.
 papermill_translators = PapermillTranslators()
 papermill_translators.register("python", PythonTranslator)
 papermill_translators.register("R", RTranslator)
 papermill_translators.register("scala", ScalaTranslator)
 papermill_translators.register("julia", JuliaTranslator)
+papermill_translators.register("ssh", SSHTranslator)


 def translate_parameters(kernel_name, parameters):
```

### 実行例

Papermillに上記パッチを当て、次のコマンドを実行します。

```shell
papermill -p HOST_A 20.20.20.20 input.ipynb output.ipynb
```